In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import os

In [10]:
DATA_DIR = '/home/jeff/Downloads/data/statoil'
train = pd.read_json(os.path.join(DATA_DIR, 'train.json'))
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)
y_train = train['is_iceberg']

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [3]:
test = pd.read_json(os.path.join(DATA_DIR, 'test.json'))
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [20]:
# note: https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712
# was used as reference; however, this version is much simpler because it doesn't use
# kfold concepts
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
x = base_model.get_layer('block5_pool').output
    

x = GlobalMaxPooling2D()(x)
x = Dense(512, activation='relu', name='fc2')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu', name='fc3')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
sgd = SGD()
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [14]:
data_aug = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, width_shift_range=0,
                              height_shift_range=0, channel_shift_range=0, zoom_range = 0.5,
                              rotation_range = 15)
es = EarlyStopping('val_loss', patience=10)
save = ModelCheckpoint(os.path.join(DATA_DIR, 'checkpoint'))

In [19]:
model.fit_generator(data_aug.flow(X_train, y_train), epochs=100, steps_per_epoch=64, verbose=True,
                   validation_data=(X_val, y_val), callbacks=[es, save])

Epoch 1/100
64/64 [==============================] - 5s - loss: 0.8144 - acc: 0.5223 - val_loss: 0.6142 - val_acc: 0.5483
Epoch 2/100
64/64 [==============================] - 3s - loss: 0.5827 - acc: 0.6812 - val_loss: 0.4972 - val_acc: 0.7913
Epoch 3/100
64/64 [==============================] - 3s - loss: 0.4828 - acc: 0.7524 - val_loss: 0.4202 - val_acc: 0.7819
Epoch 4/100
64/64 [==============================] - 3s - loss: 0.4037 - acc: 0.8170 - val_loss: 0.3727 - val_acc: 0.8318
Epoch 5/100
64/64 [==============================] - 3s - loss: 0.3657 - acc: 0.8247 - val_loss: 0.2930 - val_acc: 0.8629
Epoch 6/100
64/64 [==============================] - 3s - loss: 0.3391 - acc: 0.8378 - val_loss: 0.4093 - val_acc: 0.8474
Epoch 7/100
64/64 [==============================] - 3s - loss: 0.3613 - acc: 0.8403 - val_loss: 0.2891 - val_acc: 0.8660
Epoch 8/100
64/64 [==============================] - 3s - loss: 0.3014 - acc: 0.8618 - val_loss: 0.2669 - val_acc: 0.8910
Epoch 9/100
64/64 [=====

In [25]:
preds = model.predict(X_test)

In [26]:
with open(os.path.join(DATA_DIR, 'predictions.csv'), 'w') as pred_file:
    pred_file.write('id,is_iceberg\n')
    for i,pred in enumerate(preds):
        pred_file.write('{},{}\n'.format(test['id'][i], pred[0]))